In [2]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping


2023-11-05 11:24:19.892522: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 11:24:19.894310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 11:24:19.934563: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 11:24:19.935623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 11:24:21.159688: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
dataset = '../data/spamDetection.csv'

In [3]:
df = pd.read_csv(dataset)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df = df.rename(columns={'v1': 'target', 'v2': 'message'})
df.head()

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.drop_duplicates(inplace=True)

In [4]:
def cleanText(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text    

# df['message'] = df['message'].apply(cleanText)
# df.head()

In [7]:
df['message'] = df['message'].apply(cleanText)

In [6]:
cv = CountVectorizer()
# X = cv.fit_transform(df['message'])
# y = df['target']

# le = LabelEncoder()
# y = le.fit_transform(y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
model = Sequential()

model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=100, batch_size=32, validation_data=(X_test.toarray(), y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

y_pred = model.predict(X_test.toarray())
y_pred = np.round(y_pred).astype(int)

print(f'Accuracy score is {accuracy_score(y_test, y_pred)}')

2023-11-05 11:12:56.969009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-11-05 11:12:56.969048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: n00b
2023-11-05 11:12:56.969055: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: n00b
2023-11-05 11:12:56.969212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-05 11:12:56.969237: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-05 11:12:56.969244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 535.129.3 -- cannot find working devices in this configuration


Epoch 1/100
130/130 [==============================] - 2s 9ms/step - loss: 0.3015 - accuracy: 0.9281 - val_loss: 0.1170 - val_accuracy: 0.9768
Epoch 2/100
130/130 [==============================] - 1s 6ms/step - loss: 0.0440 - accuracy: 0.9908 - val_loss: 0.0833 - val_accuracy: 0.9816
Epoch 3/100
130/130 [==============================] - 1s 6ms/step - loss: 0.0134 - accuracy: 0.9971 - val_loss: 0.0902 - val_accuracy: 0.9816
Epoch 4/100
130/130 [==============================] - 1s 6ms/step - loss: 0.0064 - accuracy: 0.9985 - val_loss: 0.0996 - val_accuracy: 0.9826
Epoch 5/100
33/33 [==============================] - 0s 2ms/step
Accuracy score is 0.9835430784123911


In [10]:
model.save('../models/spamDetection.h5')
pickle.dump(cv, open('../models/cv.pkl', 'wb'))